In [60]:
import pureml
from pureml.decorators import dataset,load_data,transformer,model
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import pureml
from nltk import WordNetLemmatizer, pos_tag
import re, string
from nltk.tokenize import TweetTokenizer
import pickle

In [29]:
@load_data()
def load_dataset():
    df_raw = pd.read_csv('data.csv', encoding="ISO-8859-1", header=None)
    df_raw.columns = ["label", "time", "date", "query", "username", "text"]
    df = df_raw[['label', 'text']]
    df_pos = df[df['label'] == 4]
    df_neg = df[df['label'] == 0]
    df_pos = df_pos.iloc[:int(len(df_pos) / 2048)]
    df_neg = df_neg.iloc[:int(len(df_neg) / 2048)]
    df = pd.concat([df_pos, df_neg])
    return df

df = load_dataset()

In [ ]:
df

In [32]:
X = df['text']
Y = df['label']
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=0,stratify=Y)

In [ ]:
type(y_train)

In [ ]:
# @transformer()
# def read_data(x_train,y_train):
#     tk = TweetTokenizer(reduce_len=True)
#     data = []
#     x_train = x_train.tolist()
#     y_train = y_train.tolist()
#     for x, y in zip(x_train, y_train):
#         # if y == 4:
#         #     data.append((tk.tokenize(x), 4))
#         # else:
#         #     data.append((tk.tokenize(x), 0))
#         data.append((tk.tokenize(x),y))
#     return data

#data = read_data(x_train,y_train)

In [ ]:
data[4]

In [33]:
# @transformer()
def read_data(x_train):
    tk = TweetTokenizer(reduce_len=True)
    data = []
    x_train = x_train.tolist()
    # y_train = y_train.tolist()
    for i,x in enumerate(x_train):
        # if y == 4:
        #     data.append((tk.tokenize(x), 4))
        # else:
        #     data.append((tk.tokenize(x), 0))
        
        data.append((tk.tokenize(x),i))
    return data

data = read_data(x_train)

In [ ]:
data 

In [34]:
# def read_data_fun():
#     return read_data(x_train,y_train)
# %pip install dill
import dill

In [35]:
with open('utils/read_data.pickle','wb') as f:
    dill.dump(read_data,f)

In [36]:
from nltk.corpus import stopwords
STOP_WORDS = stopwords.words('english')


def cleaned(token):
    if token == 'u':
        return 'you'
    if token == 'r':
        return 'are'
    if token == 'some1':
        return 'someone'
    if token == 'yrs':
        return 'years'
    if token == 'hrs':
        return 'hours'
    if token == 'mins':
        return 'minutes'
    if token == 'secs':
        return 'seconds'
    if token == 'pls' or token == 'plz':
        return 'please'
    if token == '2morow':
        return 'tomorrow'
    if token == '2day':
        return 'today'
    if token == '4got' or token == '4gotten':
        return 'forget'
    if token == 'amp' or token == 'quot' or token == 'lt' or token == 'gt' or token == '½25':
        return ''
    return token


In [37]:
with open('utils/cleaned.pickle','wb') as f:
    dill.dump(cleaned,f)

In [58]:
#@transformer()
def list_to_dict(cleaned_tokens):
    return dict([token, True] for token in cleaned_tokens)

In [59]:
with open('utils/list_to_dict.pickle','wb') as f:
    dill.dump(list_to_dict,f)

In [40]:
#@transformer()
def remove_noise(tweet_tokens):
    cleaned_tokens = []
    for token, tag in pos_tag(tweet_tokens):
        # Eliminating the token if it is a link
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        # Eliminating the token if it is a mention
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        cleaned_token = cleaned(token.lower())
        if cleaned_token not in string.punctuation and len(cleaned_token) > 2 and cleaned_token not in STOP_WORDS:
            cleaned_tokens.append(cleaned_token)
    return cleaned_tokens

In [45]:
with open('utils/remove_noise.pickle','wb') as f:
    dill.dump(remove_noise,f)

In [46]:
#@transformer()
def cleaned_token_list(data):
    final_data = []
    for tokens, label in data:
        final_data.append((list_to_dict(tokens), label))
    print("Final Data created")
    cleaned_tokens_list = []
    for tokens, label in final_data:
        cleaned_tokens_list.append((remove_noise(tokens), label))
    return cleaned_tokens_list
    # for i,tokens in enumerate(data):
    # #     final_data.append((list_to_dict(tokens),i))
    # cleaned_token_list = []
    # for i,tokens in enumerate(final_data):
    #     cleaned_token_list.append((remove_noise(tokens),i))
    # return cleaned_token_list

# cleaned_token_list_data = cleaned_token_list(data)

In [ ]:
#cleaned_token_list_data

In [ ]:
#cleaned_token_list_data
# for i, tk_lb in enumerate(cleaned_token_list_data):
#     token,label = tk_lb
#     print(label) 


In [47]:
import dill
with open('utils/cleaned_token_list.pickle','wb') as f:
    dill.dump(cleaned_token_list,f)

In [44]:
#@transformer()
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)

        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map
#
words_to_index,index_to_words,word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

In [48]:
with open('utils/read_glove_vecs.pickle','wb') as f:
    dill.dump([words_to_index,index_to_words,word_to_vec_map],f)

In [49]:
#@transformer()
def cleared(word):
    res = ""
    prev = None
    for char in word:
        if char == prev: continue
        prev = char
        res += char
    return res

In [50]:
with open('utils/cleared.pickle','wb') as f:
    dill.dump(cleared,f)

In [51]:
#@transformer()
def sentence_to_indices(sentence_words, word_to_index, max_len, i,X):
    unks = []
    UNKS = []
    sentence_indices = []
    for j, w in enumerate(sentence_words):
        try:
            index = word_to_index[w]
        except:
            UNKS.append(w)
            w = cleared(w)
            try:
                index = word_to_index[w]
            except:
                index = word_to_index['unk']
                unks.append(w)
        X[i, j] = index

In [52]:
with open('utils/sentence_to_indices.pickle','wb') as f:
    dill.dump(sentence_to_indices,f)

In [ ]:
df = load_dataset()
print(f"DF Created. {len(df)}")
X = df['text']
Y = df['label']
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=0,stratify=Y)
data = read_data(x_train)
print(f"Data Created: {len(data)}")
cleaned_tokens_list = cleaned_token_list(data)
print(f"Cleaned_token_list Created. {len(cleaned_tokens_list)}")
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')
print(f"Read Gloves Created. {word_to_index['hello']}")
list_len = [len(i) for i, j in cleaned_tokens_list]
max_len = max(list_len)
print(f"max_len. {max_len}")
X = np.zeros((len(cleaned_tokens_list), max_len))
Y = np.zeros((len(cleaned_tokens_list), ))
print(f"X & Y Created. {len(X)} & {len(Y)}")
for i, tk_lb in enumerate(cleaned_tokens_list):
    tokens, label = tk_lb
    sentence_to_indices(tokens, word_to_index, max_len, i,X)
    #Y[i] = label
print(f"{len(X)} & {len(Y)}")
x_train = X

# x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0, stratify=Y)
print(f"{len(x_train)} & {len(x_test)} & {len(y_train)} & {len(y_test)}")

In [ ]:
y_test.shape

In [ ]:
y_train

In [ ]:
from sklearn.model_selection import train_test_split


@dataset(label='nlpexample_testing:development',upload=True)
def create_dataset():
    df = load_dataset()
    print(f"DF Created. {len(df)}")
    X = df['text']
    Y = df['label']
    x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=0,stratify=Y)
    data = read_data(x_train)
    print(f"Data Created: {len(data)}")
    cleaned_tokens_list = cleaned_token_list(data)
    print(f"Cleaned_token_list Created. {len(cleaned_tokens_list)}")
    word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')
    print(f"Read Gloves Created. {word_to_index['hello']}")
    list_len = [len(i) for i, j in cleaned_tokens_list]
    max_len = max(list_len)
    print(f"max_len. {max_len}")
    X = np.zeros((len(cleaned_tokens_list), max_len))
    Y = np.zeros((len(cleaned_tokens_list), ))
    print(f"X & Y Created. {len(X)} & {len(Y)}")
    for i, tk_lb in enumerate(cleaned_tokens_list):
        tokens, label = tk_lb
        sentence_to_indices(tokens, word_to_index, max_len, i,X)
        #Y[i] = label
    print(f"{len(X)} & {len(Y)}")
    x_train = X
    # x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0, stratify=Y)
    print(f"{len(x_train)} & {len(x_test)} & {len(y_train)} & {len(y_test)}")
    return {"x_train":x_train,"x_test":x_test,"y_train":y_train,"y_test":y_test,"max_len":max_len,"word_to_index":word_to_index," index_to_word": index_to_word,"word_to_vec_map": word_to_vec_map}

create_dataset()

In [55]:
df = pureml.dataset.fetch(label='nlpexample_testing:development:v1')

In [56]:
max_len = df['max_len']
max_len

18

In [57]:
with open('utils/max_len.pickle','wb') as f:
    dill.dump(max_len,f)

In [8]:
y_test = df['y_test']

In [9]:
y_test = df['y_test']

In [10]:
y_test

800195    4
800131    4
197       0
800225    4
91        0
         ..
183       0
163       0
800320    4
314       0
800330    4
Name: label, Length: 156, dtype: int64

In [11]:
x_train = df['x_train']

In [12]:
x_train.shape

(624, 18)

In [13]:
y_train = df['y_train']

In [14]:
y_train.shape

(624,)

In [15]:
x_test = df['x_test']

In [16]:
x_test

800195    @ben_mayer better be. I don't wanna have to co...
800131    Just spoke with my fam in #Japan via #Skype! L...
197                          Is not going to sleep tonite. 
800225    @KourtneyKardash yep Mornings are the Best! ni...
91              wednesday my b-day! don't know what 2 do!! 
                                ...                        
183       Downloading NIN's new album &quot;the slip&quo...
163       @RyanSeacrest is it just me, or she hates anoo...
800320    just discovered tonight that her church is ser...
314       My thoughts are with Sandra Cantu's family at ...
800330    ahhh loved this sunny day....I am working a go...
Name: text, Length: 156, dtype: object

In [17]:
import keras
from keras import Sequential
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Embedding


2023-06-10 18:14:31.515724: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-10 18:14:31.521541: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-10 18:14:31.579475: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-10 18:14:31.580185: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-10 18:14:32.417869: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [18]:
#Function that will initialize and populate our embedding layer

def pretrained_embedding_layer(word_to_vec_map, word_to_index, max_len):
    vocab_len = len(word_to_index) + 1
    emb_dim = word_to_vec_map["unk"].shape[0] #50

    emb_matrix = np.zeros((vocab_len, emb_dim))

    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]

    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False, input_shape=(max_len,))
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])

    return embedding_layer

In [19]:
df = pureml.dataset.fetch(label='nlpexample_testing:development:v1')
x_train = df['x_train']
x_test = df['x_test']
y_train = df['y_train']
y_test = df['y_test']
# max_len  = df['max_len']
max_len = df['max_len']
print("Data Fetched")

Data Fetched


In [20]:
@model(label='nlpexample_test:model')
def create_model():

    model = Sequential()
    word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')
    model.add(pretrained_embedding_layer(word_to_vec_map, word_to_index, df['max_len']))
    model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
    model.add(Bidirectional(LSTM(units=128, return_sequences=False)))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(df['x_train'], df['y_train'], epochs = 1, batch_size = 64, shuffle=True)
    return model

create_model()

2023-06-10 18:15:31.380343: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-10 18:15:31.382466: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-10 18:15:31.383510: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

10/10 [==============================] - 6s 54ms/step - loss: -3.2387 - accuracy: 0.0128


branch_exists True

Model has been registered!

Model Version:  v3

Model label:  nlpexample_test:model:v3

No metrics are found in config
No params are found in config
No figures are found in config


In [22]:
pureml.model.fetch(label='nlpexample_test:model:v3')

2023-06-10 18:21:37.382117: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-10 18:21:37.383737: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-10 18:21:37.385338: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [61]:
pureml.predict.add(label='nlpexample_test:model:v3',paths={'predict':'predict.py'})

Predict Function has been registered!

{"status":200,"data":[{"key":"predict","data":"https://pub-cc8d623e43954c8fa75380adfd3bb762.r2.dev/model-registry/6
266a2b5-6c43-48f1-83cb-27600b10422a/models/85bd8139-8f0f-42b7-b057-7ab207e1324d/684cc807-7573-4595-98ac-626c638c5c0
9/logs/predict_9YPFA4pbPL.txt","model_version":{"uuid":"76268dae-6319-47c2-a41d-edd136520a84","version":"v3"},"data
set_version":{"uuid":"00000000-0000-0000-0000-000000000000","version":""}}],"message":"Logs created"}

In [62]:
pureml.predict.fetch(label='nlpexample_test:model:v3')

 predict file predict.py has been fetched

In [63]:
pureml.eval(task_type='classification',
            label_dataset='nlpexample_testing:development:v1',
            label_model='nlpexample_test:model:v3')

 Succesfully fetched the dataset

 Succesfully fetched the predictor

2023-06-10 20:56:43.665202: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-10 20:56:43.667703: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-10 20:56:43.669039: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

 Succesfully fetched the model

Final Data created


2023-06-10 20:56:49.273434: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-10 20:56:49.274752: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-10 20:56:49.275771: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

5/5 [==============================] - 1s 19ms/step


{'accuracy': 0.038461538461538464,
 'precision': 0.038461538461538464,
 'recall': 0.038461538461538464,
 'f1': 0.038461538461538464,
 'confusion_matrix': array([[ 6., 72.,  0.],
        [ 0.,  0.,  0.],
        [ 9., 69.,  0.]])}